### Import Libaries

In [5]:
from serpapi import GoogleSearch
from datetime import date,timedelta,datetime
import numpy as np
import pandas as pd
import time
import sys
pd.set_option('display.max_columns', None)

### Load Dataset

In [ ]:
airports = pd.read_csv('major_international_airports_clean.csv')
temp = airports[airports['IATA_Code'].isin(['YYZ','CDG','JFK','HND'])]
airports = airports[:len(airports)-20] # take N  rows
airports = pd.concat([airports,temp], ignore_index=True).drop_duplicates().reset_index(drop=True)
display(airports)

,IATA_Code,Airport_Name,City,Country
0,YYZ,Toronto Pearson International Airport,Toronto,Canada
1,YVR,Vancouver International Airport,Vancouver,Canada
2,YUL,Montréal-Trudeau International Airport,Montreal,Canada
3,YEG,Edmonto International Airport,Edmonto,Canada
4,YYC,Calgary International Airport,Calgary,Canada
5,JFK,John F. Kennedy International Airport,New York,USA
6,LGA,LaGuardia Airport,New York,USA
7,EWR,Newark Liberty International Airport,Newark,USA
8,LAX,Los Angeles International Airport,Los Angeles,USA
9,SFO,San Francisco International Airport,San Francisco,USA


### Fight Data Extractor

In [10]:
flghDta = pd.DataFrame(columns=[
        'apt_name_dp','apt_IATA_dp','apt_time_dt_dp','apt_time_tm_dp',
        'apt_name_ds','apt_IATA_ds','apt_time_dt_ds','apt_time_tm_ds',
        'trip_class','price','ticket_sold_by','travel_type'])
def dataFightExt(data):
    for fd in data:
        fligths = fd['flights'][0]
      
        dp,ds= fligths['departure_airport'],fligths['arrival_airport']
        dp_name,ds_name = dp['name'],ds['name']
        dp_IATA,ds_IATA = dp['id'],ds['id'] 

        dp_dttm,ds_dttm = dp['time'],ds['time']
        dp_dttm_obj = datetime.strptime(dp_dttm,'%Y-%m-%d %H:%M')
        ds_dttm_obj = datetime.strptime(ds_dttm,'%Y-%m-%d %H:%M')
        dp_dt,dp_tm = dp_dttm_obj.date(),dp_dttm_obj.time()
        ds_dt,ds_tm = ds_dttm_obj.date(),ds_dttm_obj.time()

        trip_class = fligths['travel_class']
        try:
            tck_sld_by = fligths['ticket_also_sold_by']
        except:
            tck_sld_by = []

        price = fd['price']
        travel_type = fd['type']
        flghDta.loc[len(flghDta)] = [
            dp_name,dp_IATA,dp_dt,dp_tm,
            ds_name,ds_IATA,ds_dt,ds_tm,
            trip_class,price,tck_sld_by,travel_type]

### SerpAPI for Future Flight Data

In [ ]:
i = -1 # When issuess happen set this to the last value/completed here to resume after fixing said issuess with code to mange unexpected errors
outbound = date(2025,11,29)
EndPoint = date(2026,2,15)
des = ['YYZ','YYC','YEG','YUL','YVR']
Limit = False
for idx,dep in enumerate(airports['IATA_Code'].values):
  
  if idx < i: continue

  hold = ''
  if dep in des: hold = des.pop(des.index(dep))
  
  while outbound <= EndPoint:
    print(f"{idx} || {outbound}//{EndPoint} || DEP: {dep} || DES: {des}", end='\r')

    params = {
      "api_key": "ASK_FOR_KEY",
      "engine": "google_flights",
      "hl": "en",
      "gl": "ca",
      "departure_id": dep,
      "arrival_id": ','.join(des),
      "outbound_date": str(outbound),
      "currency": "USD",
      "sort_by": "3",
      "type": "2",
      "stops": "1"
    }

    chkact = GoogleSearch(params).get_account()
    if chkact['this_month_usage'] > 4890:
      Limit = True
      break
  
    search = GoogleSearch(params)
    results = search.get_dict()
    try:
      dataFightExt(results['other_flights'])
    except Exception as e:
      print(e)

    outbound += timedelta(days=7)
    
    sys.stdout.flush()
    time.sleep(0.1)

  # print(f"")

  if Limit == True: break

  if hold != '': des.append(dep)
  
  outbound = date(2025,11,29)
  
flghDta = flghDta.convert_dtypes()
display(flghDta.head(5))

'price'26-02-14//2026-02-15 || DEP: LGA || DES: ['YYZ', 'YVR', 'YUL', 'YEG', 'YYC']
'other_flights'9//2026-02-15 || DEP: ORY || DES: ['YYZ', 'YVR', 'YUL', 'YEG', 'YYC']
'other_flights'6//2026-02-15 || DEP: ORY || DES: ['YYZ', 'YVR', 'YUL', 'YEG', 'YYC']
'other_flights'3//2026-02-15 || DEP: ORY || DES: ['YYZ', 'YVR', 'YUL', 'YEG', 'YYC']
'other_flights'0//2026-02-15 || DEP: ORY || DES: ['YYZ', 'YVR', 'YUL', 'YEG', 'YYC']
'other_flights'7//2026-02-15 || DEP: ORY || DES: ['YYZ', 'YVR', 'YUL', 'YEG', 'YYC']
'other_flights'3//2026-02-15 || DEP: ORY || DES: ['YYZ', 'YVR', 'YUL', 'YEG', 'YYC']
'other_flights'0//2026-02-15 || DEP: ORY || DES: ['YYZ', 'YVR', 'YUL', 'YEG', 'YYC']
'other_flights'7//2026-02-15 || DEP: ORY || DES: ['YYZ', 'YVR', 'YUL', 'YEG', 'YYC']
'other_flights'4//2026-02-15 || DEP: ORY || DES: ['YYZ', 'YVR', 'YUL', 'YEG', 'YYC']
'other_flights'1//2026-02-15 || DEP: ORY || DES: ['YYZ', 'YVR', 'YUL', 'YEG', 'YYC']
'other_flights'7//2026-02-15 || DEP: ORY || DES: ['YYZ', 'YVR', 'Y

,apt_name_dp,apt_IATA_dp,apt_time_dt_dp,apt_time_tm_dp,apt_name_ds,apt_IATA_ds,apt_time_dt_ds,apt_time_tm_ds,trip_class,price,ticket_sold_by,travel_type
0,Toronto Pearson International Airport,YYZ,2025-11-29,00:30:00,Montréal-Pierre Elliott Trudeau International ...,YUL,2025-11-29,01:53:00,Economy,212,[],One way
1,Toronto Pearson International Airport,YYZ,2025-11-29,06:20:00,Montréal-Pierre Elliott Trudeau International ...,YUL,2025-11-29,07:48:00,Economy,212,[],One way
2,Toronto Pearson International Airport,YYZ,2025-11-29,06:30:00,Vancouver International Airport,YVR,2025-11-29,09:04:00,Economy,109,[],One way
3,Toronto Pearson International Airport,YYZ,2025-11-29,06:30:00,Calgary International Airport,YYC,2025-11-29,09:05:00,Economy,289,[],One way
4,Toronto Pearson International Airport,YYZ,2025-11-29,07:00:00,Montréal-Pierre Elliott Trudeau International ...,YUL,2025-11-29,08:23:00,Economy,159,[],One way


### SerpAPI for Hotel Deals

In [13]:
output_path = "flight_paths_nc.csv"
flghDta.to_csv(output_path, index=False)

In [12]:
flghDta.shape

(5049, 12)